## Transfer Learning using Xception to Predict Intended Gender of Watch

In [1]:

import os

import numpy as np
import pandas as pd

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dropout, Flatten, Dense,GlobalAveragePooling2D
from keras import applications
#from keras import backend as K
from keras import callbacks

Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 299, 299

# dimensions of our images.
img_width, img_height = 299, 299
n_features = 1

train_data_dir = 'multi_class_testing/train/'
validation_data_dir = 'multi_class_testing/test/'

available_train_files = len(os.listdir(train_data_dir + 'female/')) \
    + len(os.listdir(train_data_dir + 'male/'))
available_test_files = len(os.listdir(validation_data_dir + 'female/')) \
    + len(os.listdir(validation_data_dir+'male/'))
    
nb_train_samples  = available_train_files - available_train_files % 8
nb_validation_samples = available_test_files - available_test_files % 8

batch_size = 8

In [3]:
available_train_files % 8

0

Create Generators

In [4]:
datagen = ImageDataGenerator(
        rescale=1. / 255,
        rotation_range=90,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    shuffle=True,
    class_mode = 'binary')

test_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    shuffle=True,
    class_mode = 'binary')

Found 7368 images belonging to 2 classes.
Found 3152 images belonging to 2 classes.


Load Standard Xception Model

In [11]:
model = applications.InceptionV3(include_top=False, weights='imagenet')

Build My "Custom" Model

In [12]:
watch_model = Sequential()
watch_model.add(model)
watch_model.add(GlobalAveragePooling2D(name='avg_pool'))
watch_model.add(Dense(1, activation="sigmoid"))
watch_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, None, None, 2048)  21802784  
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2049      
Total params: 21,804,833
Trainable params: 21,770,401
Non-trainable params: 34,432
_________________________________________________________________


In [13]:
watch_model.layers[0].summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_95 (Conv2D)               (None, None, None, 32 864         input_2[0][0]                    
____________________________________________________________________________________________________
batch_normalization_95 (BatchNor (None, None, None, 32 96          conv2d_95[0][0]                  
____________________________________________________________________________________________________
activation_95 (Activation)       (None, None, None, 32 0           batch_normalization_95[0][0]     
___________________________________________________________________________________________

average_pooling2d_17 (AveragePoo (None, None, None, 12 0           mixed8[0][0]                     
____________________________________________________________________________________________________
conv2d_171 (Conv2D)              (None, None, None, 32 409600      mixed8[0][0]                     
____________________________________________________________________________________________________
batch_normalization_173 (BatchNo (None, None, None, 38 1152        conv2d_173[0][0]                 
____________________________________________________________________________________________________
batch_normalization_174 (BatchNo (None, None, None, 38 1152        conv2d_174[0][0]                 
____________________________________________________________________________________________________
batch_normalization_177 (BatchNo (None, None, None, 38 1152        conv2d_177[0][0]                 
___________________________________________________________________________________________

Freeze Exception Layers

In [14]:
for layer in watch_model.layers[0].layers:
    layer.trainable = False

Compile

In [15]:
watch_model.compile(
    loss = "binary_crossentropy", 
    optimizer='sgd', 
    metrics=["binary_accuracy"])

In [20]:
#Fit 

epochs = 10

cb1 = [callbacks.ModelCheckpoint(
    'inceptionv3_binary_frozen_best.h5',
    monitor='val_loss',
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False, 
    mode='auto', period=1)]




# fine-tune the model
watch_model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data = test_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks = cb1)

watch_model.save('inceptionv3_binary_frozen_final.h5')

NameError: name 'watch_model' is not defined

In [17]:
del watch_model

In [21]:
watch_model = load_model('inceptionv3_binary_frozen_final.h5')

In [22]:
watch_model.layers[0].summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_95 (Conv2D)               (None, None, None, 32 864         input_2[0][0]                    
____________________________________________________________________________________________________
batch_normalization_95 (BatchNor (None, None, None, 32 96          conv2d_95[0][0]                  
____________________________________________________________________________________________________
activation_95 (Activation)       (None, None, None, 32 0           batch_normalization_95[0][0]     
___________________________________________________________________________________________

In [25]:
#Unfreeze a few more layers and allow to run

# watch_model.layers[0].summary()

# Freeze convolutional layers
# for layer in watch_model.layers[0].layers[-3:]:
#     layer.trainable = True

    
for layer in watch_model.layers[0].layers[:249]:
    layer.trainable = False
    
for layer in watch_model.layers[0].layers[249:]:
    layer.trainable = True    
    
    
    
    
epochs = 20


watch_model.compile(
    loss = "binary_crossentropy", 
    optimizer='sgd', 
    metrics=["binary_accuracy"])

# watch_model.layers[0].summary()

cb2 = [callbacks.ModelCheckpoint(
    'inceptionv3_binary_unfrozen_convos1_best.h5',
    monitor='val_loss',
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False, 
    mode='auto', period=1)]

# fine-tune the model
watch_model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data = test_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks = cb2)


watch_model.save('inceptionv3_binary_unfrozen_convos1_final.h5')

Epoch 1/20
 83/921 [=>............................] - ETA: 275s - loss: 0.3134 - binary_accuracy: 0.8765

KeyboardInterrupt: 

In [ ]:
#del watch_model

In [5]:
watch_model = load_model('inceptionv3_binary_unfrozen_convos1_final.h5')

In [6]:
#Unfreeze a few more layers and allow to run

# watch_model.layers[0].summary()

# Freeze convolutional layers
# for layer in watch_model.layers[0].layers[-3:]:
#     layer.trainable = True

    
for layer in watch_model.layers[0].layers[:249]:
    layer.trainable = False
    
for layer in watch_model.layers[0].layers[249:]:
    layer.trainable = True    
    
    
    
    
epochs = 20


watch_model.compile(
    loss = "binary_crossentropy", 
    optimizer='sgd', 
    metrics=["binary_accuracy"])

# watch_model.layers[0].summary()

cb2 = [callbacks.ModelCheckpoint(
    'inceptionv3_binary_unfrozen_convos2_best.h5',
    monitor='val_loss',
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False, 
    mode='auto', period=1)]

# fine-tune the model
watch_model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data = test_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks = cb2)


watch_model.save('inceptionv3_binary_unfrozen_convos2_final.h5')

Epoch 1/20
921/921 [==============================] - 387s - loss: 0.1419 - binary_accuracy: 0.9434 - val_loss: 0.1836 - val_binary_accuracy: 0.9331
Epoch 2/20
921/921 [==============================] - 384s - loss: 0.1259 - binary_accuracy: 0.9511 - val_loss: 0.1445 - val_binary_accuracy: 0.9438
Epoch 3/20
921/921 [==============================] - 384s - loss: 0.1290 - binary_accuracy: 0.9496 - val_loss: 0.1441 - val_binary_accuracy: 0.9499
Epoch 4/20
921/921 [==============================] - 383s - loss: 0.1216 - binary_accuracy: 0.9539 - val_loss: 0.1725 - val_binary_accuracy: 0.9365
Epoch 5/20
921/921 [==============================] - 383s - loss: 0.1141 - binary_accuracy: 0.9539 - val_loss: 0.1523 - val_binary_accuracy: 0.9470
Epoch 6/20
921/921 [==============================] - 383s - loss: 0.1118 - binary_accuracy: 0.9575 - val_loss: 0.1475 - val_binary_accuracy: 0.9489
Epoch 7/20
921/921 [==============================] - 383s - loss: 0.1035 - binary_accuracy: 0.9601 - val_